In [1]:
import pandas as pd

In [2]:
import tkinter as tk
from tkinter import filedialog

In [3]:
root = tk.Tk()
root.withdraw()
file_path = filedialog.askopenfilename()

In [4]:
df = pd.read_excel(file_path, sheet_name="出貨明細")

In [5]:
df = df[(df["狀態"].str.contains("出")) & (df["預交日期"] <= "2021-01-31")]

In [6]:
monthly_sales = df.groupby([pd.Grouper(key="預交日期", freq="M"), "負責業務"])[["數量", "本國幣別NTD"]].sum().reset_index()

In [7]:
monthly_sales

,預交日期,負責業務,數量,本國幣別NTD
0,2019-01-31,鄭里緗,17968,1.689303e+06
1,2019-02-28,鄭里緗,9398,1.286704e+06
2,2019-03-31,鄭里緗,1998,2.208131e+06
3,2019-04-30,鄭里緗,1540,2.317719e+06
4,2019-05-31,鄭里緗,30745,2.353247e+06
5,2019-06-30,鄭里緗,78062,5.938173e+06
6,2019-07-31,鄭里緗,168548,5.601951e+06
7,2019-08-31,鄭里緗,53040,7.284139e+06
8,2019-09-30,鄭里緗,144406,5.987232e+06
9,2019-10-31,鄭里緗,142804,3.920104e+06


In [13]:
format_dict = {"預交日期": "{:%Y-%m}", "數量": "{0:,.0f}", "本國幣別NTD": "{0:,.0f}"} # 設定樣式字典來套用
test = monthly_sales.style.format(format_dict).hide_index().background_gradient(subset=["數量", "本國幣別NTD"], cmap="BuGn")

In [16]:
monthly_sales.style.format(format_dict).hide_index().bar(color="lightgreen", vmin=100_000, subset=["本國幣別NTD"]).set_caption("2019年至今各月統計表")

預交日期,負責業務,數量,本國幣別NTD
2019-01,鄭里緗,"17,968","1,689,303"
2019-02,鄭里緗,"9,398","1,286,704"
2019-03,鄭里緗,"1,998","2,208,131"
2019-04,鄭里緗,"1,540","2,317,719"
2019-05,鄭里緗,"30,745","2,353,247"
2019-06,鄭里緗,"78,062","5,938,173"
2019-07,鄭里緗,"168,548","5,601,951"
2019-08,鄭里緗,"53,040","7,284,139"
2019-09,鄭里緗,"144,406","5,987,232"
2019-10,鄭里緗,"142,804","3,920,104"


In [20]:
month_order = ["January", "February", "March", "April",
              "May", "June", "July", "August",
              "September", "October", "November", "December"]
df["預交月份"] = pd.Categorical(df["預交月份"], ordered=True, categories=month_order)

In [ ]:
result = test.data # .data將styler轉回dataframe才能進行其他操作，例如作圖之類的

## Pivot table轉回DataFrame方式為
---
1. 使用.to_records()將數據轉為array
2. 在用pd.DataFrame的方式將array放進去即可

In [26]:
pd.DataFrame(pd.pivot_table(df, index=["預交年份", "負責業務"], values="本國幣別NTD", columns="預交月份").to_records())

,預交年份,負責業務,January,February,March,April,May,June,July,August,September,October,November,December
0,2019,鄭里緗,49685.378522,41506.584064,76142.460999,74765.130880,138426.307049,164949.247467,186731.706210,280159.195495,171063.780016,280007.438354,77690.848400,56802.838840
1,2020,墨漢雷迪,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53624.589352,26061.098667,27145.000000,133200.960000,158621.024416
2,2020,許凱智,9895.884546,15066.658731,11660.572268,9838.243409,9642.611110,20577.593254,17008.130445,34864.562651,32995.781797,27496.040068,34271.474316,40749.022627
3,2020,鄭里緗,97024.197926,59783.244042,80313.210277,73526.817917,94262.683744,90378.073982,115573.371070,120383.140037,109250.488824,139054.804195,148111.620297,91566.630044
4,2021,墨漢雷迪,291367.594800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2021,許凱智,63928.873640,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2021,鄭里緗,96538.900548,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
import plotly.express as px

In [ ]:
px.line(result, x="預交日期", y="本國幣別NTD", color="負責業務", 
        title="業績走勢圖", labels={"本國幣別NTD": "營業額", "預交日期": "月份"}).show()

In [ ]:
test.to_excel(r"C:\Users\kaihsu\Desktop\test.xlsx", index=False)